In [4]:
import numpy as np
import PIL.Image

import torch, torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from matplotlib import pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import KFold

import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Imprime se vai ser CPU ou alguma GPU
print(f'{device}\n')
image_size = 48
numero_de_epocas = 20
bs = 32
dataset = r'./data/Fer-2013/'
pasta_treino = os.path.join(dataset, 'treino')
pasta_validacao = os.path.join(dataset, 'validacao')
pasta_teste = os.path.join(dataset, 'teste')
numero_de_classes = len(os.listdir(pasta_validacao))

from tqdm import tqdm

definicoes_de_transformacoes = {
    'treino': transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.2),
        transforms.RandomRotation(degrees=30),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),

    'validacao': transforms.Compose([
        transforms.Grayscale(num_output_channels=3),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
}

transformacoes_de_imagens = definicoes_de_transformacoes


data = {
    'treino': datasets.ImageFolder(root=pasta_treino, transform=transformacoes_de_imagens['treino']),
    'validacao': datasets.ImageFolder(root=pasta_validacao, transform=transformacoes_de_imagens['validacao'])
}

indice_para_classe = {v: k for k, v in data['treino'].class_to_idx.items()}
print(indice_para_classe)

num_imagens_treino = len(data['treino'])
num_imagens_validacao = len(data['validacao'])

train_labels = [label for _, label in data['treino'].imgs]
class_weights = compute_class_weight('balanced', classes=np.arange(numero_de_classes), y=train_labels)
weights = torch.tensor(class_weights, dtype=torch.float).to(device)

data_loader_treino = DataLoader(data['treino'], batch_size=bs, shuffle=True)
data_loader_validacao = DataLoader(data['validacao'], batch_size=bs, shuffle=True)

resnet50 = models.resnet50(pretrained=True)

for param in list(resnet50.parameters())[:-10]:
    param.requires_grad = True

num_features = resnet50.fc.in_features
resnet50.fc = nn.Sequential(
    nn.Linear(num_features, 1024),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(1024, 512),
    nn.ReLU(),
    nn.Dropout(0.4),
    nn.Linear(512, numero_de_classes),
    nn.LogSoftmax(dim=1)
)

funcao_erro = nn.CrossEntropyLoss(weight=weights)
otimizador = optim.AdamW(resnet50.parameters(), lr=0.0001)
scheduler = ReduceLROnPlateau(otimizador, mode='min', factor=0.1, patience=3)


def treinar_e_validar_cross_validation(modelo, metrica_erro, otimizador_sgd, scheduler, epocas=25, folds=5):
    kf = KFold(n_splits=folds, shuffle=True, random_state=42)
    fold_results = []

    for fold, (train_idx, val_idx) in enumerate(kf.split(data['treino'])):
        print(f"\nIniciando Fold {fold + 1}/{folds}")

        train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
        val_sampler = torch.utils.data.SubsetRandomSampler(val_idx)
        
        data_loader_train_fold = DataLoader(data['treino'], batch_size=bs, sampler=train_sampler)
        data_loader_val_fold = DataLoader(data['treino'], batch_size=bs, sampler=val_sampler)

        melhor_acuracia = 0.0
        melhor_modelo = None

        for epoca in range(epocas):
            inicio_epoca = time.time()
            print(f"\nFold {fold + 1}, Época {epoca + 1}/{epocas}")

            # Treinamento
            modelo.train()
            erro_treino = 0.0
            acuracia_treino = 0.0
            total_treino = 0

            for entradas, labels in tqdm(data_loader_train_fold, desc="Treinando"):
                entradas = entradas.to(device)
                labels = labels.to(device)

                otimizador_sgd.zero_grad()
                saidas = modelo(entradas)
                erro = metrica_erro(saidas, labels)
                erro.backward()
                otimizador_sgd.step()

                erro_treino += erro.item() * entradas.size(0)

                _, preds = torch.max(saidas, 1)
                acuracia_treino += torch.sum(preds == labels).item()
                total_treino += labels.size(0)

            erro_medio_treino = erro_treino / len(train_idx)
            acuracia_treino = acuracia_treino / total_treino

            # Validação
            modelo.eval()
            erro_validacao = 0.0
            acuracia_validacao = 0.0
            total_validacao = 0

            with torch.no_grad():
                for entradas, labels in tqdm(data_loader_val_fold, desc="Validando"):
                    entradas = entradas.to(device)
                    labels = labels.to(device)
                    saidas = modelo(entradas)
                    erro = metrica_erro(saidas, labels)
                    erro_validacao += erro.item() * entradas.size(0)

                    _, preds = torch.max(saidas, 1)
                    acuracia_validacao += torch.sum(preds == labels).item()
                    total_validacao += labels.size(0)

                erro_medio_validacao = erro_validacao / len(val_idx)
                acuracia_validacao = acuracia_validacao / total_validacao

            print(f"\nFold {fold + 1}, Época {epoca + 1}:")
            print(f"Erro Treino: {erro_medio_treino:.4f}, Acurácia Treino: {acuracia_treino:.4f}")
            print(f"Erro Validação: {erro_medio_validacao:.4f}, Acurácia Validação: {acuracia_validacao:.4f}")

            # Salvar o melhor modelo baseado na validação
            if acuracia_validacao > melhor_acuracia:
                melhor_acuracia = acuracia_validacao
                melhor_modelo = modelo.state_dict()
                torch.save(melhor_modelo, f'./melhores_modelos/melhor_modelo_fold_{fold + 1}.pt')

            scheduler.step(erro_medio_validacao)

        fold_results.append(acuracia_validacao)

    print(f"\nAcurácia Média Validação (Cross-Validation): {np.mean(fold_results):.4f}")

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f'{device}\n')

numero_de_epocas = 20

resnet50 = resnet50.to(device)
treinar_e_validar_cross_validation(resnet50, funcao_erro, otimizador, scheduler, numero_de_epocas, folds=5)


cuda:0

{0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}


c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Danil\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


cuda:0


Iniciando Fold 1/5

Fold 1, Época 1/20


Validando: 100%|██████████| 180/180 [00:11<00:00, 16.27it/s]



Fold 1, Época 1:
Erro Treino: 1.7341, Acurácia Treino: 0.3255
Erro Validação: 1.5831, Acurácia Validação: 0.4117

Fold 1, Época 2/20


...


Fold 5, Época 18:
Erro Treino: 0.7829, Acurácia Treino: 0.6995
Erro Validação: 0.6543, Acurácia Validação: 0.7372

Fold 5, Época 19/20


Validando: 100%|██████████| 180/180 [00:10<00:00, 16.49it/s]



Fold 5, Época 19:
Erro Treino: 0.7827, Acurácia Treino: 0.7006
Erro Validação: 0.6452, Acurácia Validação: 0.7478

Fold 5, Época 20/20


Validando: 100%|██████████| 180/180 [00:10<00:00, 16.74it/s]


Fold 5, Época 20:
Erro Treino: 0.7761, Acurácia Treino: 0.6997
Erro Validação: 0.6624, Acurácia Validação: 0.7507

Acurácia Média Validação (Cross-Validation): 0.6913
